In [1]:
# 43679 -- Interactive Visualization
# 2025 - 2026
# 2nd semester
# Lab 1 - EDA (independent)
# ver 1.1
# 24022026 - Added questions at end; cleaning

## Lab 01<br>Task 3: Independent EDA and Cleaning

The purpose of this task is for you to practice EDA for a new dataset in a more independent manner. Feel free to go back to Task 2's code and reuse it, whenever it makes sense. Nevertheless, **don't limit yourself to just copy-pasting** and undersstand why you are applying each step. Understanding what are the issues and how to address them will be important for your final project.

**Dataset:** `dataset_D_git_classroom_activity.csv`

---

### Context

You have been handed an activity log from a Git-based classroom platform. It records **10,000 events** -- commits, pull requests, CI runs, code reviews, and test runs -- generated by students and bots across multiple repositories.

Your goal is to apply the same EDA and cleaning pipeline from Task 2 to this new dataset. This time the guidance is lighter: each section tells you *what* to look for and *which tools and methods to use*, but the code is yours to write.

### Pipeline reminder

| Step | Tool | Goal |
|---|---|---|
| 1 — Load and inspect | pandas | Understand structure and inferred types |
| 2 — Automated profiling | SweetViz | Triage issues across all columns |
| 3 — Navigate and inspect | D-Tale | See problems with your own eyes |
| 4 — Clean | pandas | Fix each issue with explicit, reproducible code |
| 5 — Verify | D-Tale + SweetViz | Confirm fixes landed correctly |

---

## Part 1: Load and Inspect

In [2]:
import pandas as pd
import sweetviz as sv
import dtale
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('dataset_D_git_classroom_activity.csv')

# Inspect shape, column types, and first rows
# Use: df.shape, df.dtypes, df.head()


> **What to note:** Which columns were inferred as `object` but should be boolean or numeric? Any column that should be numeric but is `object` almost always signals a formatting problem in the raw values.

---

## Part 2: Automated Profiling with SweetViz

Generate a SweetViz report on the raw dataset. Use it to fill in the triage checklist below before moving on.

In [ ]:
# Generate the SweetViz report
# Use: sv.analyze(df)
# Save to 'sweetviz_git_raw.html'


### Triage checklist

| Question | Your finding |
|---|---|
| Which columns have missing values? Which has the most, and by how much? | *...* |
| Which columns are shown as TEXT but should be boolean? | *...* |
| Which columns are shown as TEXT but should be numeric? | *...* |
| How many distinct values does `event_type` have? Does that seem right? | *...* |
| What is unusual about `ci_status` distinct values compared to `event_type`? | *...* |
| Are there numeric columns with suspicious ranges? | *...* |

*(Double-click to fill in your answers)*

---

## Part 3: Navigate and Inspect with D-Tale

Launch D-Tale and use it to confirm each issue visually. Do not clean anything here.

In [ ]:
# Launch D-Tale
# Use: dtale.show(df, host='127.0.0.1', subprocess=False, open_browser=False)


### Inspection checklist

For each item, use D-Tale's **column header → Describe** to inspect value counts and distribution.

| What to inspect | What you should find |
|---|---|
| `is_weekend` unique values | 8 representations of True/False |
| `event_type` unique values | Many case/whitespace variants of 7 event types |
| `ci_status` unique values | Case/whitespace variants — but also: are FAILED and FAILURE the same thing? |
| `os` unique values | WIN, Windows, win — which is the canonical form? |
| `coverage_percent` raw values | Some use comma as decimal separator |
| `pr_merge_time_hours` missing % | Very high — is this random or structural? |
| `tests_failed` vs `tests_run` | Sort `tests_failed` descending — are there rows where it exceeds `tests_run`? |
| `lines_added` distribution | Any extreme values? |
| `pr_merge_time_hours` min | Any negative values? |
| `commit_message_length` min | Any zero values? What would a zero-length commit message mean? |

<br>

> **Note on `pr_merge_time_hours`:** Think carefully about why this column has so many missing values before deciding what to do. Look at the `event_type` column for rows where it is missing -- does a pattern emerge?

*(Record any additional observations below)*

---

## Part 4: Clean with Pandas

Work through each issue below. For each one: **inspect --> fix --> verify**.  
The first example in each category is more detailed; subsequent columns follow the same pattern.

Start by creating a working copy:

In [ ]:
df_clean = df.copy()

---

### 4.1. Boolean columns

**Columns:** `is_weekend`, `label_is_high_quality`, `exam_period`  
**Issue:** 8 different representations of True/False  
**Approach:** `.map()` with an explicit dictionary, same as Task 2  

> **Hint:** Define the `bool_map` dictionary once and reuse it for all three columns. Include both string and boolean keys to make the mapping safe to re-run.

In [ ]:
# Inspect
print(sorted(df_clean['is_weekend'].dropna().unique().tolist()))

In [ ]:
# Fix is_weekend, label_is_high_quality, exam_period
# Your code here


In [ ]:
# Verify — each column should have only True and False, 0 nulls
for col in ['is_weekend', 'label_is_high_quality', 'exam_period']:
    print(f"{col}: {df_clean[col].value_counts().to_dict()} | nulls: {df_clean[col].isna().sum()}")

---

### 4.2. `is_bot_user`: case and whitespace

**Issue:** 6 variants of 2 values (`Human`, `Bot`) with mixed case and whitespace  
**Approach:** `.str.strip().str.lower()` — no typos, no synonym merging needed

In [ ]:
# Inspect
print(df_clean['is_bot_user'].value_counts().to_string())

In [ ]:
# Fix is_bot_user
# Your code here


In [ ]:
# Verify — should show exactly 2 values: 'human' and 'bot'
print(df_clean['is_bot_user'].value_counts())

---

### 4.3. Categorical columns: case and whitespace

**Columns:** `dominant_language`, `editor`, `os`, `event_type`  
**Issue:** Many case/whitespace variants — strip and lowercase resolves most  

> **Note on `os`:** After stripping and lowercasing you will still have `win` and `windows` as separate values. Decide on a canonical form and merge them with `.replace()`.

> **Note on `event_type`:** After stripping and lowercasing, verify the number of unique values matches the number of distinct event types you expect.

In [ ]:
# Inspect dominant_language before
print(f'dominant_language unique before: {df_clean["dominant_language"].nunique()}')

In [ ]:
# Fix dominant_language — strip and lowercase
# Your code here

# Apply the same to editor and event_type
# Your code here

# Fix os — strip, lowercase, then merge win/windows variants
# Your code here


In [ ]:
# Verify
for col in ['dominant_language', 'editor', 'os', 'event_type']:
    print(f"{col} ({df_clean[col].nunique()} unique): {sorted(df_clean[col].dropna().unique().tolist())}")

---

### 4.4. `ci_status`: case, whitespace, and synonym merging

**Issue:** Case and whitespace variants — but also `FAILED` and `FAILURE` represent the same outcome and need to be merged into one canonical value.  
**Approach:** Strip and lowercase first, then use `.replace()` to merge synonyms.

> **Decision to make:** After lowercasing, you will have `failed` and `failure` as separate values. Pick one as the canonical form and justify your choice in a markdown cell below.

In [ ]:
# Inspect
print(df_clean['ci_status'].value_counts().to_string())

In [ ]:
# Fix ci_status — strip, lowercase, then merge synonyms
# You can use .replace({'current':'replaced'})
# Your code here


In [ ]:
# Verify — should show exactly 4 values: success, failed, cancelled + your merged form
print(df_clean['ci_status'].value_counts())

> **Your decision:** Which canonical form did you choose for `failed`/`failure`, and why? This is where you need to go for the domain context. What is the common term?

*(Double-click to write your answer)*

---

### 4.5. `coverage_percent`: comma decimal separator and type conversion

**Issue:** Loaded as `object` — some values use a comma instead of a decimal point  
**Approach:** Same as `purchase_amount` in Task 2 — `.str.replace()` then `.astype(float)`

In [ ]:
# Inspect — how many rows have a comma?
print(df_clean['coverage_percent'].dtype)
comma_rows = df_clean['coverage_percent'].astype(str).str.contains(',', na=False)
print(f'Rows with comma: {comma_rows.sum()}')

# tip: any values outside the valid range? 
# What is the valid range for this variable?

In [ ]:
# Fix coverage_percent
# Your code here


In [ ]:
# Verify

print(f'dtype: {df_clean["coverage_percent"].dtype}')
print(df_clean['coverage_percent'].describe().round(2))
print(f'\nValues < 0:   {(df_clean["coverage_percent"] < 0).sum()} rows')
print(f'Values > 100: {(df_clean["coverage_percent"] > 100).sum()} rows')

---

### 4.6. Missing values: decisions and strategy

This dataset has four columns with missing values. Inspect each one and decide what to do.

| Column | Missing | Your hypothesis for why | Your decision |
|---|---|---|---|
| `pr_merge_time_hours` | 71.7% | *...* | *...* |
| `commit_message_length` | 7.0% | *...* | *...* |
| `build_duration_s` | 2.1% | *...* | *...* |
| `time_to_ci_minutes` | 2.0% | *...* | *...* |

*(Double-click to fill in the table)*

> **Hint for `pr_merge_time_hours`:** Filter D-Tale to show only rows where `pr_merge_time_hours` is NOT null. What values appear in `event_type`? What does this tell you about why it is missing for the other rows?

In [ ]:
# Inspect missing counts
missing = df_clean.isnull().sum()
pct = (missing / len(df_clean) * 100).round(1)
pd.DataFrame({'missing': missing, '%': pct})[missing > 0]

In [ ]:
# Investigate pr_merge_time_hours — which event types have non-null values?
print(df_clean.loc[df_clean['pr_merge_time_hours'].notna(), 'event_type'].value_counts())

In [ ]:
# Apply your decisions from the table above
# Your code here


---

### 4.7. Outliers and impossible values

Three issues to address:

**A. `pr_merge_time_hours` — negative values**  
A negative merge time is impossible. Inspect the affected rows and set them to `NaN`.  
Use: boolean mask + `.loc[mask, col] = float('nan')`

**B. `tests_failed > tests_run` — cross-column logical impossibility**  
231 rows have more failed tests than tests were run — physically impossible. This is a new type of issue: it requires checking consistency *between* two columns, not just inspecting one in isolation.  
Inspect the affected rows, then set `tests_failed` to `NaN` for those rows.

**C. `lines_added` and `lines_deleted` — extreme outliers**  
Some commits add or delete thousands of lines — potentially valid (e.g. adding a large library) or a logging error.  
Inspect the affected rows before deciding. Document your threshold choice.

In [ ]:
# A — Inspect negative pr_merge_time_hours
neg_mask = df_clean['pr_merge_time_hours'] < 0
print(f'Negative pr_merge_time_hours: {neg_mask.sum()}')
print(df_clean.loc[neg_mask, ['event_type', 'pr_merge_time_hours']].head())

In [ ]:
# Fix A — set negative values to NaN
# Your code here


In [ ]:
# B — Inspect tests_failed > tests_run
impossible_mask = df_clean['tests_failed'] > df_clean['tests_run']
print(f'Rows where tests_failed > tests_run: {impossible_mask.sum()}')
print(df_clean.loc[impossible_mask, ['tests_run', 'tests_failed']].describe().round(1))

In [ ]:
# Fix B — set tests_failed to NaN for impossible rows
# Your code here


In [ ]:
# C — Inspect lines_added and lines_deleted outliers
print('lines_added distribution:')
print(df_clean['lines_added'].describe().round(1))
print(f'\nRows > 1000 lines added: {(df_clean["lines_added"] > 1000).sum()}')
print(df_clean.loc[df_clean['lines_added'] > 1000, 
                   ['event_type', 'lines_added', 'lines_deleted', 'dominant_language']].head(8).to_string())

In [ ]:
# Fix C — apply your decision on lines_added and lines_deleted outliers
# Your code here


> **Your decisions:** What thresholds did you use? What was your reasoning for each?

*(Double-click to write your answers)*

---

### 4.8. **OPTIONAL** `timestamp`: mixed datetime formats 

Like Task 2, the `timestamp` column contains mixed datetime formats. However, unlike Task 2, there is no derived column that depends on it — so the impact of unresolved timestamps is lower here.

Apply a first-pass parse with `pd.to_datetime(utc=True, errors='coerce')`. Check how many rows remain unparsed. If you want to go further, apply the `try_formats()` strategy from Task 2's optional section.

In [ ]:
# Parse timestamp — first pass
# Your code here


---

## Part 5: Verify with D-Tale

In [ ]:
# Reload D-Tale with the cleaned dataframe
# Use: dtale.show(df_clean, host='127.0.0.1', subprocess=False, open_browser=False)


Check each of the following in D-Tale:

| Column | Expected result |
|---|---|
| `is_weekend`, `label_is_high_quality`, `exam_period` | Only `True` / `False` |
| `is_bot_user` | Only `human` / `bot` |
| `event_type` | Exactly 7 values, all lowercase |
| `ci_status` | Exactly 4 values, no `failure`/`FAILED` duplicates |
| `os` | Exactly 3 values, no `win`/`windows` duplicates |
| `coverage_percent` | dtype = float64 |
| `pr_merge_time_hours` | No negative values |
| `tests_failed` | No values exceeding `tests_run` |

---

## Part 6: Before vs After with SweetViz

In [ ]:
# Generate comparison report
# Exclude timestamp if you converted it (same reason as Task 2)
# Save to 'sweetviz_git_comparison.html'
# Your code here


---

## Part 7: Save

In [ ]:
df_clean.to_csv('dataset_D_git_classroom_activity_clean.csv', index=False)
print(f'Saved: {len(df_clean)} rows, {len(df_clean.columns)} columns')

---

## Final Questions

Answer the following before finishing:

**1.** The `pr_merge_time_hours` column is missing for 71.7% of rows. Is this a data quality problem? Why or why not?

**2.** You found rows where `tests_failed > tests_run`. What does this kind of cross-column check tell you that a single-column inspection would have missed?

**3.** For `ci_status`, you had to decide whether `failed` and `failure` are the same thing. What kind of knowledge -- beyond the data itself -- did you need to make that decision?

**4.** Compare this dataset to the telemetry dataset from Task 2. Which issues were the same? Which were new? What does that tell you about the generality of the cleaning skills you are building?
